In [1]:
## PREPROCESS
# chia dataset ra
# thêm negative là dự đoán của bge-m3

## HYBRID SEARCH
# dùng BM25 để lấy top 50
# tạo Dataset class phù hợp vói triplet loss
# fine tune bi-encoder PhoBERT triplet loss với LoRA và lấy top 30
# hợp nhất và lấy top 40

## RERANK
# tạo Dataset class phù hợp vói binary classification
# fine tune cross encoder PhoBERT với binary classification

# EVAL
%env CUDA_LAUNCH_BLOCKING=1


env: CUDA_LAUNCH_BLOCKING=1


# PREPROCESS

In [2]:
import pandas as pd
corpus = pd.read_csv("/kaggle/input/retrieval/corpus.csv")
train = pd.read_csv("/kaggle/input/retrieval/train.csv")

## DATASET

In [3]:
import pandas as pd
import re

def prepare_data(train_df, corpus_df, sample_size=500, max_corpus_size=10000, output_prefix="hybrid_search", random_state=42):
    # Bước 1: Sample dữ liệu
    train_sampled = train_df.sample(n=sample_size, random_state=random_state)

    # Bước 2: Làm sạch cột cid
    def robust_clean_cid_string(s):
        if not isinstance(s, str):
            return []
        s = s.replace('[', '').replace(']', '').replace("'", '').replace('"', '').strip()
        s = re.sub(r'\s+', ',', s)
        try:
            return [int(x) for x in s.split(',') if x.strip().isdigit()]
        except Exception as e:
            print(f"⚠️ Lỗi parse cid: {s} → {e}")
            return []

    train_sampled['cid'] = train_sampled['cid'].apply(robust_clean_cid_string)

    # Bước 3: Lọc các sample có cid hợp lệ
    available_cids = set(corpus_df['cid'].unique())
    train_sampled = train_sampled[train_sampled['cid'].apply(lambda cids: all(cid in available_cids for cid in cids))]

    # Bước 4: Tạo positive_context
    all_cids = set(cid for sublist in train_sampled['cid'] for cid in sublist)
    cid_to_text = corpus_df.set_index("cid")['text'].to_dict()
    train_sampled['positive_context'] = train_sampled['cid'].apply(
        lambda cids: "\n".join([cid_to_text.get(cid, "") for cid in cids if cid in cid_to_text])
    )

    # Bước 5: Lọc lại corpus
    corpus_filtered = corpus_df[corpus_df['cid'].isin(all_cids)].copy()
    corpus_sampled = corpus_filtered.sample(n=1000, random_state=42) if len(corpus_filtered) > max_corpus_size else corpus_filtered

    # Bước 6: Clean và lưu
    train_sampled = train_sampled.dropna(subset=['question', 'positive_context'])
    train_sampled.to_csv(f"train_{output_prefix}.csv", index=False)
    corpus_sampled.to_csv(f"corpus_{output_prefix}.csv", index=False)

    print(f"✅ Saved: {len(train_sampled)} training rows and {len(corpus_sampled)} corpus rows.")


prepare_data(train, corpus, sample_size=500, output_prefix="hybrid_search", random_state=123)
prepare_data(train, corpus, sample_size=500, output_prefix="rerank", random_state=456)

✅ Saved: 497 training rows and 526 corpus rows.
✅ Saved: 499 training rows and 546 corpus rows.


In [4]:
import pandas as pd
import ast

def check_cid_consistency(train_path, corpus_path):
    train_df = pd.read_csv(train_path)
    corpus_df = pd.read_csv(corpus_path)

    # Chuẩn hóa cột cid từ string → list[int]
    def parse_cid(x):
        if isinstance(x, list):
            return x
        try:
            return ast.literal_eval(x)
        except:
            return []

    train_df['cid'] = train_df['cid'].apply(parse_cid)

    # Tập hợp tất cả cid xuất hiện trong train
    train_cids = set(cid for sublist in train_df['cid'] for cid in sublist)

    # Tập hợp toàn bộ cid trong corpus
    corpus_cids = set(corpus_df['cid'].unique())

    # So sánh
    missing_cids = train_cids - corpus_cids

    if not missing_cids:
        print("✅ Tất cả CID trong train đều có trong corpus.")
    else:
        print(f"❌ Có {len(missing_cids)} CID bị thiếu trong corpus:")
    return missing_cids
check_cid_consistency(train_path="train_rerank.csv", corpus_path="corpus_rerank.csv")
check_cid_consistency(train_path="train_hybrid_search.csv", corpus_path="corpus_hybrid_search.csv")

✅ Tất cả CID trong train đều có trong corpus.
✅ Tất cả CID trong train đều có trong corpus.


set()

## thêm hard negative bằng dự đoán của bge-m3

In [5]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

def generate_hard_negatives_with_bge(
    train_file: str,
    corpus_file: str,
    output_file: str = "train_neg_hybrid_search.csv",
    model_name: str = "BAAI/bge-m3",
    batch_size: int = 64,
    max_length: int = 256
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load model & tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    # Load data
    train_sampled = pd.read_csv(train_file)
    corpus_sampled = pd.read_csv(corpus_file)

    # Lấy text corpus và map cid
    cid_to_text = corpus_sampled.set_index("cid")['text'].to_dict()
    all_corpus_texts = list(cid_to_text.values())
    all_corpus_cids = list(cid_to_text.keys())

    # Hàm encode
    def get_embedding(texts):
        inputs = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt').to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            return outputs.last_hidden_state[:, 0].cpu().numpy()  # [CLS]

    # Embed corpus
    print("🔄 Encoding corpus...")
    corpus_embeddings = []
    for i in tqdm(range(0, len(all_corpus_texts), batch_size)):
        batch_texts = all_corpus_texts[i:i+batch_size]
        batch_emb = get_embedding(batch_texts)
        corpus_embeddings.append(batch_emb)
    corpus_embeddings = np.vstack(corpus_embeddings)

    # Tạo hard negatives
    print("🔍 Generating hard negatives...")
    negatives = []

    for idx, row in tqdm(train_sampled.iterrows(), total=len(train_sampled)):
        question = row['question']
        pos_cids = set(row['cid']) if isinstance(row['cid'], list) else set()

        q_emb = get_embedding([question])
        sims = cosine_similarity(q_emb, corpus_embeddings)[0]
        top_indices = sims.argsort()[::-1]

        selected_neg = None
        for top_idx in top_indices:
            neg_cid = all_corpus_cids[top_idx]
            if neg_cid not in pos_cids:
                selected_neg = all_corpus_texts[top_idx]
                break

        negatives.append(selected_neg or "no hard negative found")

    # Gán và lưu
    train_sampled['negative'] = negatives
    train_sampled = train_sampled.dropna(subset=['question', 'positive_context', 'negative'])
    train_sampled = train_sampled.drop(columns=['positive_context'])
    train_sampled.to_csv(output_file, index=False)

    print(f"✅ Đã tạo negative và lưu: {output_file} ({len(train_sampled)} samples)")
    return train_sampled


In [6]:
generate_hard_negatives_with_bge(
    train_file="train_hybrid_search.csv",
    corpus_file="corpus_hybrid_search.csv",
    output_file="train_neg_hybrid_search.csv"
)

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

100%|██████████| 9/9 [00:13<00:00,  1.49s/it]


🔍 Generating hard negatives...


100%|██████████| 497/497 [00:15<00:00, 32.03it/s]


✅ Đã tạo negative và lưu: train_neg_hybrid_search.csv (497 samples)


,question,context,cid,qid,negative
0,Người vi phạm quy định về phòng cháy chữa cháy...,"['Tội vi phạm quy định về phòng cháy, chữa chá...",[61969],124743,"""Điều 122. Xử lý vi phạm pháp luật về bảo hiểm..."
1,Chi nhánh của tổ chức hòa giải thương mại nước...,"['Quyền và nghĩa vụ của chi nhánh, văn phòng đ...",[31484],24462,"Quyền và nghĩa vụ của chi nhánh, văn phòng đại..."
2,Tài xế dương tính ma túy điều khiển phương tiệ...,"['Điều 82. Tạm giữ phương tiện, giấy tờ có liê...",[61478],81790,"""Điều 82. Tạm giữ phương tiện, giấy tờ có liên..."
3,Ủy ban nhân dân có thẩm quyền hoạt động về thẩ...,['Thẩm quyền quản lý nhà nước về thẩm định giá...,[185298],110549,Thẩm quyền quản lý nhà nước về thẩm định giá\n...
4,Người bán hàng chèo kéo khách ở khu vực lễ hội...,['Vi phạm quy định về tổ chức lễ hội\n1. Phạt ...,[41036],142434,Vi phạm quy định về tổ chức lễ hội\n1. Phạt cả...
...,...,...,...,...,...
492,Việc quản lý thuế được thực hiện dựa trên nhữn...,"['Nguyên tắc quản lý thuế\n1. Mọi tổ chức, hộ ...",[40812],158272,"""Điều 5. Nguyên tắc quản lý thuế\n1. Mọi tổ ch..."
493,Viện Môi trường nông nghiệp có chức năng gì?,['Vị trí chức năng\nViện Môi trường nông nghiệ...,[82441],18758,Vị trí chức năng\nViện Môi trường nông nghiệp ...
494,"Có được chứng thực chữ ký trong giấy mua, bán,...",['Trường hợp không được chứng thực chữ ký\n1. ...,"[32178, 61946]",30805,"""Điều 2. Giải thích từ ngữ\nTrong Nghị định nà..."
495,Thẩm quyền giải quyết ly hôn thuận tình trong ...,"['""Điều 28. Những tranh chấp về hôn nhân và gi...","[234931, 62484, 114145]",154951,"Quyền và nghĩa vụ của chi nhánh, văn phòng đại..."


In [7]:
generate_hard_negatives_with_bge(
    train_file="train_rerank.csv",
    corpus_file="corpus_rerank.csv",
    output_file="train_neg_rerank.csv"
)

🔄 Encoding corpus...


100%|██████████| 9/9 [00:13<00:00,  1.47s/it]


🔍 Generating hard negatives...


100%|██████████| 499/499 [00:14<00:00, 33.97it/s]


✅ Đã tạo negative và lưu: train_neg_rerank.csv (499 samples)


,question,context,cid,qid,negative
0,Bình đẳng giới trong gia đình được thể hiện qu...,"['Bình đẳng giới trong gia đình\n1. Vợ, chồng ...",[53911],121967,"Bình đẳng giới trong gia đình\n1. Vợ, chồng bì..."
1,Khiến người đang ở trong tình trạng quẫn bách ...,['Tội cưỡng dâm\n1. Người nào dùng mọi thủ đoạ...,"[67438, 67438]",122529,"""Điều 141. Tội hiếp dâm\n1. Người nào dùng vũ ..."
2,Hiệp hội công chứng viên Việt Nam có những nhi...,['Nhiệm vụ và quyền hạn của Hiệp hội công chứn...,[36322],104501,Nhiệm vụ và quyền hạn của Hiệp hội công chứng ...
3,Hồ sơ đề nghị điều chỉnh nội dung giấy phép ho...,"['Hồ sơ đề nghị gia hạn, điều chỉnh nội dung g...",[68054],18818,"Hồ sơ đề nghị gia hạn, điều chỉnh nội dung giấ..."
4,Thời hiệu xử phạt vi phạm hành chính đối với c...,['Thời hiệu xử lý vi phạm hành chính\n1. Thời ...,[61543],46122,"""Điều 6. Thời hiệu xử lý vi phạm hành chính\n1..."
...,...,...,...,...,...
494,Thời gian hưởng phụ cấp trách nhiệm đối với Ch...,['Phụ cấp đối với quân nhân dự bị đã xếp vào đ...,[55239],74445,"""Điều 3. Phụ cấp đối với quân nhân dự bị đã xế..."
495,Hồ sơ đề nghị cấp Giấy phép trang bị vũ khí qu...,['THỦ TỤC HÀNH CHÍNH MỚI BAN HÀNH\nA. Thủ tục ...,[71771],18899,THỦ TỤC HÀNH CHÍNH MỚI BAN HÀNH\nA. Thủ tục hà...
496,Những người chưa học đến lớp 8 nhưng muốn tham...,['Tiêu chuẩn tuyển quân\n...\n4. Tiêu chuẩn vă...,[74470],69044,"""Điều 4. Tiêu chuẩn tuyển quân\n...\n4. Tiêu c..."
497,Cơ quan thuế địa phương có trách nhiệm gì tron...,['Trách nhiệm của cơ quan thuế địa phương\nCơ ...,[117519],50074,Trách nhiệm của cơ quan thuế địa phương\nCơ qu...


In [8]:

# Load file
df = pd.read_csv("train_neg_rerank.csv")  # hoặc tên file bạn đang dùng

# Tạo 2 bản ghi cho mỗi dòng: 1 positive và 1 negative
pos_rows = df[["question", "context"]].copy()
pos_rows["label"] = 1
pos_rows = pos_rows.rename(columns={"context": "context"})

neg_rows = df[["question", "negative"]].copy()
neg_rows["label"] = 0
neg_rows = neg_rows.rename(columns={"negative": "context"})

# Gộp lại
rerank_df = pd.concat([pos_rows, neg_rows], ignore_index=True)

# Shuffle để tránh model bias
rerank_df = rerank_df.sample(frac=1.0, random_state=42).reset_index(drop=True)

# Lưu
rerank_df.to_csv("rerank_train.csv", index=False)
print("✅ Saved rerank_train.csv with shape:", rerank_df.shape)


✅ Saved rerank_train.csv with shape: (998, 3)


# HYBRID SEARCH

## BM25 để lấy top 20

In [9]:
!pip install rank_bm25

In [10]:
from rank_bm25 import BM25Okapi
def retrieve_bm25(query, corpus, top_k=20):
    corpus = pd.read_csv(corpus)
    bm25 = BM25Okapi(corpus["text"].str.split())
    scores = bm25.get_scores(query.split())
    top_idx = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
    return corpus.iloc[top_idx]

## Bi-encoder để lấy top 10

In [11]:
from torch.utils.data import Dataset
import pandas as pd

class LegalBiEncoderDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length):
        self.data = pd.read_csv(file_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        question = row['question']
        pos = row['context']
        neg = row['negative']

        anchor = self.tokenizer(question, padding='max_length', truncation=True,
                                max_length=self.max_length, return_tensors="pt")
        positive = self.tokenizer(pos, padding='max_length', truncation=True,
                                  max_length=self.max_length, return_tensors="pt")
        negative = self.tokenizer(neg, padding='max_length', truncation=True,
                                  max_length=self.max_length, return_tensors="pt")

        return {
            "anchor_input_ids": anchor["input_ids"].squeeze(),
            "anchor_attention_mask": anchor["attention_mask"].squeeze(),
            "positive_input_ids": positive["input_ids"].squeeze(),
            "positive_attention_mask": positive["attention_mask"].squeeze(),
            "negative_input_ids": negative["input_ids"].squeeze(),
            "negative_attention_mask": negative["attention_mask"].squeeze()
        }


In [12]:
import torch
import torch.nn as nn
from transformers import AutoModel
from peft import get_peft_model, LoraConfig, TaskType

class BiEncoderPhoBERT(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        base_model = AutoModel.from_pretrained(model_name)

        # LoRA config
        peft_config = LoraConfig(
            task_type=TaskType.FEATURE_EXTRACTION,
            r=8,
            lora_alpha=16,
            lora_dropout=0.1,
            target_modules=["encoder.layer.11.attention.output.dense"],  # hoặc mở rộng thêm
            bias="none"
        )
        self.encoder = get_peft_model(base_model, peft_config)

    def forward(self, input_ids, attention_mask):
        output = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = output.last_hidden_state[:, 0]  # [CLS] vector
        return cls_output


In [13]:
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "vinai/phobert-base"
max_length = 256
epochs = 10
batch_size = 32
lr = 2e-5


tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
dataset = LegalBiEncoderDataset("/kaggle/working/train_neg_hybrid_search.csv", tokenizer, max_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = BiEncoderPhoBERT(model_name).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
loss_fn = nn.TripletMarginLoss(margin=0.3)

# Training loop
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in tqdm(dataloader):
        anchor = model(batch['anchor_input_ids'].to(device), batch['anchor_attention_mask'].to(device))
        pos = model(batch['positive_input_ids'].to(device), batch['positive_attention_mask'].to(device))
        neg = model(batch['negative_input_ids'].to(device), batch['negative_attention_mask'].to(device))

        loss = loss_fn(anchor, pos, neg)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    
    print(f"Epoch {epoch + 1}: Loss = {total_loss:.4f}")


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

...

Epoch 8: Loss = 14.3479


100%|██████████| 16/16 [00:13<00:00,  1.16it/s]


Epoch 9: Loss = 12.6180


100%|██████████| 16/16 [00:13<00:00,  1.15it/s]

Epoch 10: Loss = 11.6262


In [14]:

import os
output_dir = "checkpoints/bi_encoder_phobert"
os.makedirs(output_dir, exist_ok=True)

model.encoder.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"✅ Đã lưu checkpoint tại {output_dir}")

✅ Đã lưu checkpoint tại checkpoints/bi_encoder_phobert


In [15]:
import torch
from transformers import AutoTokenizer, AutoModel
from peft import PeftModel, PeftConfig
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Load corpus (chứa cid, text)
corpus_df = pd.read_csv("/kaggle/working/corpus_rerank.csv")
corpus_texts = corpus_df["text"].tolist()
corpus_cids = corpus_df["cid"].tolist()

# Load model + tokenizer
def load_dense_model(checkpoint_path="checkpoints/bi_encoder_phobert"):
    config = PeftConfig.from_pretrained(checkpoint_path)
    base = AutoModel.from_pretrained(config.base_model_name_or_path)
    model = PeftModel.from_pretrained(base, checkpoint_path)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path, use_fast=False)
    return model.eval(), tokenizer

# Encode corpus (chạy một lần hoặc lưu sẵn)
def encode_corpus(model, tokenizer, device, max_len=256):
    embeddings = []
    model.eval()
    model.to(device)

    with torch.no_grad():
        for text in corpus_texts:
            inputs = tokenizer(text, truncation=True, padding='max_length', max_length=max_len, return_tensors="pt")
            inputs = {k: v.to(device) for k, v in inputs.items()}
            output = model(inputs["input_ids"], inputs["attention_mask"])
            cls_output = output.last_hidden_state[:, 0]  # [CLS] token
            emb = cls_output.cpu().numpy()[0]
            embeddings.append(emb)

    return torch.tensor(embeddings)


# Build index
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, tokenizer = load_dense_model()
corpus_embeddings = encode_corpus(model, tokenizer, device)

# Define dense_func
def dense_func(query, top_k=10):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(query, truncation=True, padding='max_length', max_length=256, return_tensors="pt").to(device)
        output = model(inputs["input_ids"], inputs["attention_mask"])
        cls_output = output.last_hidden_state[:, 0]  # lấy vector [CLS]
        query_vec = cls_output.cpu().numpy()

    sims = cosine_similarity(query_vec, corpus_embeddings.numpy())[0]
    top_indices = sims.argsort()[::-1][:top_k]

    results = []
    for idx in top_indices:
        results.append({
            "cid": corpus_cids[idx],
            "text": corpus_texts[idx],
            "score": float(sims[idx]) # chỉ để debug thôi, sau cần xóa
        })

    return results



<ipython-input-15-b8eaa6c8fd5a>:35: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(embeddings)


In [16]:
def hybrid_search(query, bm25_func, dense_func, corpus):
    bm25_res = bm25_func(query, corpus)          
    dense_res = dense_func(query)        

    # Convert BM25 từ DataFrame sang list[dict]
    bm25_list = bm25_res.to_dict(orient="records")

    # Hợp nhất + khử trùng lặp theo cid
    merged = {item["cid"]: item for item in bm25_list + dense_res}

    # đang ưu tiên bm25
    return list(merged.values())[:25]


### TESTING

In [17]:
# testing
query = "Phó Tổng Giám đốc Ngân hàng Chính sách xã hội được xếp lương theo bảng lương như thế nào?"
results = dense_func(query)
results

[{'cid': 113598,
  'text': '"Điều 37. Huy động tiền gửi và cho vay ngoại tệ trong nước \nNgân hàng Nhà nước Việt Nam quy định việc huy động, cho vay bằng ngoại tệ trên lãnh thổ Việt Nam của tổ chức tín dụng."',
  'score': 0.6749408841133118},
 {'cid': 159058,
  'text': 'Kinh phí hỗ trợ giải báo chí hàng năm\nKinh phí hỗ trợ Giải báo chí hàng năm bao gồm kinh phí hoạt động của Ban Tổ chức giải thưởng và Hội đồng xét chọn giải thưởng, kinh phí tổ chức Lễ trao giải thưởng và kinh phí phục vụ cho công tác tổ chức xét giải thưởng được bố trí từ ngân sách của Bộ KH&CN giao cho Trung tâm Nghiên cứu và Phát triển truyền thông khoa học và công nghệ hàng năm theo quy định của Luật Ngân sách nhà nước, nguồn huy động hợp pháp của các tổ chức, cá nhân và kinh phí từ Quỹ Thi đua khen thưởng của Trung tâm Nghiên cứu và Phát triển truyền thông khoa học và công nghệ.',
  'score': 0.6604733467102051},
 {'cid': 149524,
  'text': 'Vị trí và chức năng\nVụ Hợp tác quốc tế là đơn vị thuộc Bộ Tài chính, có ch

In [18]:
## testing
query = "Phó Tổng Giám đốc Ngân hàng Chính sách xã hội được xếp lương theo bảng lương như thế nào?"
corpus = "/kaggle/working/corpus_hybrid_search.csv"
bm25_results = retrieve_bm25(query,corpus)
bm25_results

,text,cid
97,"""Điều 2. Đối tượng áp dụng\nCán bộ, công chức,...",40804
369,"Nhiệm vụ, quyền hạn của Tổng Giám đốc.\n...\n8...",121167
98,Mức lương cơ sở\n1. Mức lương cơ sở dùng làm c...,40805
401,"Tổng công ty có quyền tổ chức quản lý, tổ chức...",139233
83,"""Điều 3. Mức phụ cấp công vụ\nCác đối tượng qu...",34367
294,Phân phối lợi nhuận của Công ty mẹ\nLợi nhuận ...,84755
240,"""Điều 8. Chế độ, chính sách\n...\n2. Trong thờ...",71809
165,"""Điều 90. Tiền lương\n1. Tiền lương là số tiền...",61954
370,"""II. CÁCH XẾP LƯƠNG\n1. Xếp lương khi nâng ngạ...",121338
40,"""Điều 26. Tài chính công đoàn\nTài chính công ...",16882


In [20]:
## testing
query = "Phó Tổng Giám đốc Ngân hàng Chính sách xã hội được xếp lương theo bảng lương như thế nào?"
hybrid_searching = hybrid_search(query, retrieve_bm25, dense_func, corpus )
hybrid_searching

[{'text': '"Điều 2. Đối tượng áp dụng\nCán bộ, công chức, viên chức, người lao động trong các cơ quan, tổ chức, đơn vị của Đảng, Nhà nước, tổ chức chính trị - xã hội từ trung ương đến xã, phường, thị trấn (sau đây gọi chung là cấp xã) và người hưởng lương trong lực lượng vũ trang (bao gồm cả trường hợp điều động, biệt phái, luân chuyển và không phân biệt người địa phương với người nơi khác đến) đã được xếp lương theo bảng lương do cơ quan có thẩm quyền của Đảng và Nhà nước quy định, đang công tác và đến công tác ở vùng có điều kiện kinh tế - xã hội đặc biệt khó khăn, gồm:\n1. Cán bộ, công chức, viên chức (kể cả người tập sự) trong các cơ quan, tổ chức, đơn vị sự nghiệp của Đảng, Nhà nước, tổ chức chính trị - xã hội từ trung ương đến cấp xã;\n2. Người làm việc theo chế độ hợp đồng lao động trong các cơ quan, đơn vị của Đảng, Nhà nước, tổ chức chính trị - xã hội quy định tại Nghị định số 68/2000/NĐ-CP ngày 17 tháng 11 năm 2000 của Chính phủ về thực hiện chế độ hợp đồng một số loại công v

# RERANK


In [21]:
from torch.utils.data import Dataset
import pandas as pd

class RerankDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=256):
        self.data = pd.read_csv(file_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        question = row["question"]
        context = row["context"]
        label = row["label"]  # 1 = relevant, 0 = not relevant

        encoded = self.tokenizer(
            question,
            context,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            "input_ids": encoded["input_ids"].squeeze(0),
            "attention_mask": encoded["attention_mask"].squeeze(0),
            "label": int(label)
        }


In [22]:

os.environ["WANDB_DISABLED"] = "true"
import warnings
warnings.filterwarnings("ignore", message="Be aware, overflowing tokens.*")


In [23]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

model_name = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Load dataset
df = pd.read_csv("rerank_train.csv")  # Cột: question, context, label
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

train_df.to_csv("rerank_train_split.csv", index=False)
val_df.to_csv("rerank_val_split.csv", index=False)

train_dataset = RerankDataset("rerank_train_split.csv", tokenizer)
val_dataset = RerankDataset("rerank_val_split.csv", tokenizer)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Training args
training_args = TrainingArguments(
    output_dir="./rerank_ckpt",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2
)

# Metric
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }

# Trainer
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


# Train
trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-23-fcc40f2b915c>:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Be aware, overfl

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.002162,1.000000,1.000000
2,No log,0.001019,1.000000,1.000000
3,No log,0.000833,1.000000,1.000000


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

TrainOutput(global_step=339, training_loss=0.04202640443424911, metrics={'train_runtime': 110.2741, 'train_samples_per_second': 24.43, 'train_steps_per_second': 3.074, 'total_flos': 354410591569920.0, 'train_loss': 0.04202640443424911, 'epoch': 3.0})

In [24]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

def rerank_with_cross_encoder(question, contexts, model_path, model_name="vinai/phobert-base", top_k=5, device=None, max_length=256):
    import torch
    from transformers import AutoTokenizer, AutoModelForSequenceClassification

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load model & tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device).eval()

    # ✅ Làm sạch context: loại None, cắt dài
    contexts = [c if isinstance(c, str) else "" for c in contexts]
    contexts = [c[:2048] for c in contexts]  # ngăn tokenizer tạo >512 tokens

    try:
        inputs = tokenizer(
            [question] * len(contexts),
            contexts,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        ).to(device)
    except Exception as e:
        print("❌ Tokenizer error:", e)
        print("👉 Context length:", [len(c) for c in contexts])
        raise

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)[:, 1]

    results = list(zip(contexts, probs.cpu().tolist()))
    reranked = sorted(results, key=lambda x: x[1], reverse=True)

    return reranked[:top_k]



In [25]:
best_ckpt_path = trainer.state.best_model_checkpoint
print("✅ Best checkpoint path:", best_ckpt_path)


✅ Best checkpoint path: ./rerank_ckpt/checkpoint-113


In [26]:
# Một example
question = "Phó Tổng Giám đốc Ngân hàng Chính sách xã hội được xếp lương theo bảng lương như thế nào?"
contexts = [item['text'] for item in hybrid_searching]

reranked = rerank_with_cross_encoder(
    question=question,
    contexts=contexts,
    model_path=best_ckpt_path,  # tôi không biết tên checkpoint
    model_name=model_name,
    top_k=3,
    device="cpu"
)

# In kết quả
for i, (ctx, score) in enumerate(reranked, 1):
    print(f"#{i} - score: {score:.4f}")
    print(ctx)
    print("-" * 50)


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

#1 - score: 0.0282
''Điều 4. Áp dụng mức lương tối thiểu
1. Mức lương tối thiểu tháng là mức lương thấp nhất làm cơ sở để thỏa thuận và trả lương đối với người lao động áp dụng hình thức trả lương theo tháng, bảo đảm mức lương theo công việc hoặc chức danh của người lao động làm việc đủ thời giờ làm việc bình thường trong tháng và hoàn thành định mức lao động hoặc công việc đã thỏa thuận không được thấp hơn mức lương tối thiểu tháng.
2. Mức lương tối thiểu giờ là mức lương thấp nhất làm cơ sở để thỏa thuận và trả lương đối với người lao động áp dụng hình thức trả lương theo giờ, bảo đảm mức lương theo công việc hoặc chức danh của người lao động làm việc trong một giờ và hoàn thành định mức lao động hoặc công việc đã thỏa thuận không được thấp hơn mức lương tối thiểu giờ.
3. Đối với người lao động áp dụng hình thức trả lương theo tuần hoặc theo ngày hoặc theo sản phẩm hoặc lương khoán thì mức lương của các hình thức trả lương này nếu quy đổi theo tháng hoặc theo giờ không được thấp hơn 